# Lab 3.6: Image Captioning and VQA

**Objective**: Generate captions and answer questions about images

**Duration**: 30 minutes

## Learning Outcomes
- Use BLIP for image captioning
- Perform visual question answering
- Understand multi-modal model outputs

In [ ]:
import sys
sys.path.insert(0, "../../../src")
from hf_ecosystem import __version__
print(f"hf-ecosystem version: {__version__}")

In [ ]:
from transformers import pipeline, BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import requests
from io import BytesIO

## 1. Image Captioning with BLIP

In [ ]:
# Load BLIP model and processor (manual inference since pipeline API changed in transformers 5.0)
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
print("BLIP captioning model loaded")

In [ ]:
# Load image (using HuggingFace docs which doesn't block requests)
url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg"
response = requests.get(url)
image = Image.open(BytesIO(response.content)).convert("RGB")

# Generate caption using manual inference
inputs = blip_processor(image, return_tensors="pt")
output = blip_model.generate(**inputs, max_new_tokens=50)
caption = [{"generated_text": blip_processor.decode(output[0], skip_special_tokens=True)}]
print(f"Caption: {caption[0]['generated_text']}")

## 2. Visual Question Answering

In [ ]:
vqa = pipeline("visual-question-answering", model="dandelin/vilt-b32-finetuned-vqa", device="cpu")
print("VQA model loaded")

In [ ]:
# Ask questions about the image
questions = ["What color is prominent?", "Is this a photo?"]

for q in questions:
    answer = vqa(image, q)
    print(f"Q: {q}")
    print(f"A: {answer[0]['answer']} ({answer[0]['score']:.3f})")
    print()

## Verification

In [ ]:
def verify_lab():
    assert len(caption) > 0
    assert "generated_text" in caption[0]
    print("✅ Lab completed successfully!")

verify_lab()